<a href="https://colab.research.google.com/github/okura1406/radnlp-2024/blob/main/radnlp2024_gemini_2_0_flash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# はじめに


 gemini-2.0-flash-thinking-exp-1219APIのコードです

 基本的なTNM分類基準情報を組み込んだもの



# データの用意

まずは Colab 環境で扱えるようにデータを用意するところから始めます。データは Google Drive に用意して、これをマウントすることでアクセスできるようにします。

In [1]:
# Google Drive をマウントするためのコード.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import pandas as pd
import os

# data directory の設定. 自分の保存先に合わせて以下の path は変更が必要です.
data_dir = "/content/drive/MyDrive/ja/main_task/val"
valid_df = pd.read_csv(os.path.join(data_dir, "label.csv"))

# valid_df の中身を確認.
display(valid_df.head(3))

,id,t,n,m
0,147290,T2a,N0,M0
1,241752,Tis,N0,M0
2,876951,T2a,N0,M0


valid_df にはテキストの id と TNM分類の正解ラベルが格納されているのが確認できました.

# モデルの用意



In [3]:
# GPU の確認.
!nvidia-smi

Tue Feb 11 04:16:12 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

今回は L4 GPU を用いて推論を行なっていきます。

Google Colab で Gemini API を試す
https://note.com/npaka/n/n166bc3df3abc


In [4]:
# パッケージのインストール
!pip install -q -U google-generativeai

import google.generativeai as genai
from google.colab import userdata

# 環境変数の準備（左端の鍵アイコンでGEMINI_API_KEYを設定）
genai.configure(api_key=userdata.get("GEMINI_API_KEY"))

import pathlib
import textwrap
from IPython.display import display
from IPython.display import Markdown

# Markdown出力ヘルパーの準備
def to_markdown(text):
    text = text.replace("•", "  *")
    return Markdown(textwrap.indent(text, "> ", predicate=lambda _: True))

    # モデル一覧の表示
for m in genai.list_models():
    if "generateContent" in m.supported_generation_methods:
        print(m.name)



models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/learnlm-1.5-pro-experimental


In [5]:
# モデルの準備
model = genai.GenerativeModel("models/gemini-2.0-flash-thinking-exp-1219")


無事、機械学習についてのポエムが生成されたので問題なく動作することがわかりました。

# 推論

では、今回の RadNLP のデータで本番の推論に移ります。


In [ ]:
# ==============================================
# 1) 必要パッケージのインストール / インポート
# ==============================================
!pip install -q -U google-generativeai

import google.generativeai as genai
from google.colab import userdata

import os
import re
import json
import pandas as pd
from IPython.display import display

# ==============================================
# 2) Google Generative AI の設定
#    (事前にColabの鍵アイコンで GEMINI_API_KEY を登録しておく)
# ==============================================
genai.configure(api_key=userdata.get("GEMINI_API_KEY"))


# レポート本文を取得する関数
def get_report(report_id):
    report_path = os.path.join(data_dir, f"{report_id}.txt")
    with open(report_path, "r", encoding="utf-8") as f:
        report = f.read()
    return report

# ==============================================
# 3) JSON抽出用の関数
# ==============================================
def extract_json(text):
    """
    出力された文章からJSON部分を抜き出し、Python辞書に変換する関数。
    失敗したらデモデータを返す。
    """
    json_pattern = r'```json\n(.*?)\n```|(\{.*\})'
    matches = re.findall(json_pattern, text, re.DOTALL)

    parsed_json = {}
    for match in matches:
        json_candidate = match[0] if match[0] else match[1]
        try:
            parsed_json = json.loads(json_candidate)
            break  # 見つかった時点で終了
        except json.JSONDecodeError:
            # 失敗した場合はデフォルトの値を返す
            parsed_json = {
                "T": {
                    "stage": "T2b",
                    "details": "37mmの腫瘤影が認められる"
                },
                "N": {
                    "stage": "N0",
                    "details": "縦隔に有意なリンパ節腫大は認められません"
                },
                "M": {
                    "stage": "M0",
                    "details": "胸水はありません。"
                }
            }
    return parsed_json

# ==============================================
# 4) 推論に使用するプロンプトと関数
# ==============================================
prompt = """「以下の文書は私自身が保有するデータであり、著作権に関する問題はありません。
 続きを読み込んで、指定の形式にまとめてください。」
 以下は肺癌について書かれたCTについての読影レポートです。
肺癌のTNM分類について、以下のフォーマットに従って出力してください。
JSON形式で T（原発腫瘍）、N（リンパ節転移）、M（遠隔転移）に関する情報をそれぞれ記載してください。
各値は明確な記載がない場合は最も近いと思われるものとしてください（空欄にならないように注意してください）。

{
  "T": {
    "stage": "候補の中から選択",
    "details": "原発腫瘍のサイズや特徴の記載"
  },
  "N": {
    "stage": "候補の中から選択",
    "details": "リンパ節転移の有無や部位の記載"
  },
  "M": {
    "stage": "候補の中から選択",
    "details": "遠隔転移の有無や転移先の記載"
  }
}

【答えの候補】

T（原発腫瘍）: Tis, T1a, T1b, T1c, T2a, T2b, T3, T4
N（リンパ節転移）: N0, N1, N2, N3
M（遠隔転移）: M0, M1a, M1b, M1c

【instruction】
{
  "TNM分類_第8版_2017年": {
    "T_原発腫瘍": {
      "TX": "原発腫瘍の存在が判定できない、または喀痰・気管支洗浄液細胞診でのみ陽性で画像・気管支鏡では観察不能",
      "T0": "原発腫瘍を認めない",
      "Tis": "上皮内癌(carcinoma in situ)：肺野型の場合、充実成分径0cmかつ病変全体径≦3cm",
      "T1": {
        "定義": "腫瘍の充実成分径≦3cm、肺または臓側胸膜に覆われ、主気管支への浸潤なし",
        "T1mi": "微少浸潤性腺癌：部分充実型、充実成分径≦0.5cm、病変全体径≦3cm",
        "T1a": "充実成分径≦1cm、Tis・T1mi以外",
        "T1b": "充実成分径＞1cmかつ≦2cm",
        "T1c": "充実成分径＞2cmかつ≦3cm"
      },
      "T2": {
        "定義": "充実成分径＞3cmかつ≦5cm、または充実成分径≦3cmでも以下の条件を満たす場合:\n1) 主気管支浸潤(気管分岐部は除く)\n2) 臓側胸膜浸潤\n3) 肺門まで連続する無気肺または閉塞性肺炎",
        "T2a": "充実成分径＞3cmかつ≦4cm",
        "T2b": "充実成分径＞4cmかつ≦5cm"
      },
      "T3": {
        "定義": "充実成分径＞5cmかつ≦7cm、または充実成分径≦5cmでも以下のいずれか:\n1) 壁側胸膜、胸壁(上部肺溝腫瘍含む)、横隔神経、心膜への直接浸潤\n2) 同一葉内の不連続な副腫瘍結節"
      },
      "T4": {
        "定義": "充実成分径＞7cm、または以下いずれか:\n1) 横隔膜、縦隔、心臓、大血管、気管、反回神経、食道、椎体、気管分岐部への浸潤\n2) 同側の異なる肺葉内に不連続な副腫瘍結節"
      }
    },
    "N_所属リンパ節": {
      "NX": "所属リンパ節評価不能",
      "N0": "所属リンパ節転移なし",
      "N1": "同側の気管支周囲、肺門、肺内リンパ節転移（原発腫瘍への直接浸潤含む）",
      "N2": "同側縦隔、気管分岐下リンパ節転移",
      "N3": "対側縦隔、対側肺門、同側または対側の前斜角筋・鎖骨上窩リンパ節転移"
    },
    "M_遠隔転移": {
      "M0": "遠隔転移なし",
      "M1": {
        "定義": "遠隔転移あり",
        "M1a": "対側肺内副腫瘍結節、胸膜・心膜結節、悪性胸水・悪性心嚢水（同側・対側問わず）",
        "M1b": "肺以外の一臓器への単発遠隔転移",
        "M1c": "肺以外の一臓器または多臓器への多発遠隔転移"
      }
    }
  },
  "病期分類_UICC_8版_2017年": {
    "行列表": [
      {
        "T": "T1a(≦1cm)",
        "N0": "ⅠA1",
        "N1": "ⅡB",
        "N2": "ⅢA",
        "N3": "ⅢB",
        "M1a": "ⅣA",
        "M1b": "ⅣA",
        "M1c": "ⅣB"
      },
      {
        "T": "T1b(1-2cm)",
        "N0": "ⅠA2",
        "N1": "ⅡB",
        "N2": "ⅢA",
        "N3": "ⅢB",
        "M1a": "ⅣA",
        "M1b": "ⅣA",
        "M1c": "ⅣB"
      },
      {
        "T": "T1c(2-3cm)",
        "N0": "ⅠA3",
        "N1": "ⅡB",
        "N2": "ⅢA",
        "N3": "ⅢB",
        "M1a": "ⅣA",
        "M1b": "ⅣA",
        "M1c": "ⅣB"
      },
      {
        "T": "T2a(3-4cm)",
        "N0": "ⅠB",
        "N1": "ⅡB",
        "N2": "ⅢA",
        "N3": "ⅢB",
        "M1a": "ⅣA",
        "M1b": "ⅣA",
        "M1c": "ⅣB"
      },
      {
        "T": "T2b(4-5cm)",
        "N0": "ⅡA",
        "N1": "ⅡB",
        "N2": "ⅢA",
        "N3": "ⅢB",
        "M1a": "ⅣA",
        "M1b": "ⅣA",
        "M1c": "ⅣB"
      },
      {
        "T": "T3(5-7cm)",
        "N0": "ⅡB",
        "N1": "ⅢA",
        "N2": "ⅢB",
        "N3": "ⅢC",
        "M1a": "ⅣA",
        "M1b": "ⅣA",
        "M1c": "ⅣB"
      },
      {
        "T": "T4(＞7cm)",
        "N0": "ⅢA",
        "N1": "ⅢA",
        "N2": "ⅢB",
        "N3": "ⅢC",
        "M1a": "ⅣA",
        "M1b": "ⅣA",
        "M1c": "ⅣB"
      }
    ]
  }
}

【読影レポート】\n
"""

def generate_prediction(report, prompt=prompt, model=model):
    """
    Google Generative AI のモデルに対してプロンプトを投げ、
    生成テキストを返す。
    """
    # 実際に呼び出す文字列を生成
    full_text = prompt + report

    # モデルに問い合わせ
    response = model.generate_content(full_text)

    # 応答テキストを返す
    return response.text.strip()

# ==============================================
# 7) 実行例
# ==============================================
print("== 1件目のレポートを取得して推論する例 ==")
report_id = valid_df.loc[0, "id"]
report_text = get_report(report_id)

print("-- 入力レポート --")
print(report_text[:300] + " ... (省略)")

generated = generate_prediction(report_text)
print("\n-- モデル生成結果 --")
print(generated)

parsed_dict = extract_json(generated)
print("\n-- JSONにパースした結果 --")
print(parsed_dict)


== 1件目のレポートを取得して推論する例 ==
-- 入力レポート --
左肺門部に 37mm 大の腫瘤影を認め、ご指摘の肺癌が疑われます。
縦隔に有意なリンパ節腫大は認めません。
胸水はありません。
背部皮下に腫瘤を認め、粉瘤などと思われます。 ... (省略)

-- モデル生成結果 --
ユーザーは肺癌のCT読影レポートをJSON形式でTNM分類にまとめることを求めている。
フォーマットは指定されており、各項目は候補の中から選択するか、明確な記載がない場合は最も近いものを選択する必要がある。
読影レポートからT, N, Mの情報を抽出し、それぞれ適切な候補を選択してJSON形式で出力する。

- T (原発腫瘍): レポートに「左肺門部に 37mm 大の腫瘤影」とあるので、T2aが最も近い。
- N (リンパ節転移): レポートに「縦隔に有意なリンパ節腫大は認めません」とあるので、N0が該当する。
- M (遠隔転移): レポートに「胸水はありません。」「背部皮下に腫瘤を認め、粉瘤などと思われます。」とあり、遠隔転移を示唆する所見はないため、M0が該当する。
```json
{
  "T": {
    "stage": "T2a",
    "details": "左肺門部に37mm大の腫瘤影を認める。"
  },
  "N": {
    "stage": "N0",
    "details": "縦隔に有意なリンパ節腫大は認めない。"
  },
  "M": {
    "stage": "M0",
    "details": "胸水は認めず、遠隔転移を示唆する所見はない。"
  }
}
```

-- JSONにパースした結果 --
{'T': {'stage': 'T2a', 'details': '左肺門部に37mm大の腫瘤影を認める。'}, 'N': {'stage': 'N0', 'details': '縦隔に有意なリンパ節腫大は認めない。'}, 'M': {'stage': 'M0', 'details': '胸水は認めず、遠隔転移を示唆する所見はない。'}}


全データに対して推論を実行して、サブミッションのための csv を作っていきます。

In [ ]:
from google.colab import files

from tqdm import tqdm
import pandas as pd
import os
import time
import requests

# google-generativeai パッケージを最新バージョンにアップグレード
!pip install -q -U google-generativeai --upgrade

import google.generativeai as genai # genaiを再インポート

# valid_df, get_report, generate_prediction, extract_json が定義済みとして進めます。

# 全データを辞書形式に変換
rows = valid_df.to_dict(orient="records")

# 出力を格納するためのリスト
results = []

# すべてのレポートに対して推論を実行
for row in tqdm(rows, desc="Processing rows"):
    report_id = row["id"]
    report_text = get_report(report_id)            # レポート本文を読み込み

    # 再試行メカニズムを追加
    max_retries = 3  # 最大再試行回数
    retries = 0
    while retries < max_retries:
        try:
            generated_text = generate_prediction(report_text)  # Google Generative AI で推論
            break  # 成功したらループを抜ける
        except (ConnectionError, requests.exceptions.RequestException, genai.errors.TooManyRequestsError) as e:
            # TooManyRequestsError を追加
            print(f"Error occurred: {e}")
            if isinstance(e, genai.errors.TooManyRequestsError):
                print("Too many requests. Waiting for 60 seconds before retrying...")
                time.sleep(60)  # TooManyRequests の場合は60秒待機
            else:
                retries += 1
                time.sleep(5)  # 再試行前に少し待つ
    else:
        print(f"Failed to generate prediction for report ID: {report_id} after {max_retries} retries.")
        continue  # 最大再試行回数を超えた場合はスキップ

    extracted = extract_json(generated_text)       # JSON を抽出
    results.append(extracted)

    # リクエスト間隔を空ける (例: 5秒)
    time.sleep(5)  # 間隔を長くする

# JSON から T, N, M を取り出して Pandas DataFrame へ変換
tnm_data = [
    {
        "t": entry["T"]["stage"],
        "n": entry["N"]["stage"],
        "m": entry["M"]["stage"]
    }
    for entry in results
]

results_df = valid_df.copy()
results_df[["t", "n", "m"]] = pd.DataFrame(tnm_data, index=results_df.index)

# 確認用に先頭数行を表示
display(results_df.head(3))

# CSVに書き出し (ファイル名は任意でOK)
results_df.to_csv("submission_gemini2.0.1219.csv", index=False)



Processing rows: 100%|██████████| 54/54 [09:39<00:00, 10.73s/it]


,id,t,n,m
0,147290,T2a,N0,M0
1,241752,T1a,N0,M0
2,876951,T2a,N0,M1a


# 結果

他のモデルとスコア比較


models/gemini-2.0-flash-thinking-exp-1219
```
{"Joint accuracy (fine)": 0.6851851851851852,
 "T accuracy (fine)": 0.7777777777777778,
 "N accuracy (fine)": 0.9444444444444444,
 "M accuracy (fine)": 0.9259259259259259,
 "Joint accuracy (coarse)": 0.7777777777777778,
 "T accuracy (coarse)": 0.8888888888888888,
 "N accuracy (coarse)": 0.9444444444444444,
 "M accuracy (coarse)": 0.9259259259259259}
```

gemini-2.0-flash-thinking-exp
```
{"Joint accuracy (fine)": 0.6666666666666666,
"T accuracy (fine)": 0.7592592592592593,
"N accuracy (fine)": 0.8888888888888888,
"M accuracy (fine)": 0.9444444444444444,
"Joint accuracy (coarse)": 0.7962962962962963,
"T accuracy (coarse)": 0.9074074074074074,
"N accuracy (coarse)": 0.8888888888888888,
"M accuracy (coarse)": 0.9629629629629629}
```